In [1]:
import sys 
import gymnasium as gym
from gymnasium.wrappers.record_video import RecordVideo
from gymnasium.wrappers.record_episode_statistics import RecordEpisodeStatistics
import kidspuzzles
from stable_baselines3 import A2C

n_digits = 10
# reward_clipped = -0.02
# reward_enter_target_area = 0.01
# reward_exit_target_area = -0.02

env = gym.make(
    'kidspuzzles/DigitsPuzzleEnv-v0', 
    render_mode = 'rgb_array', 
    n_digits = n_digits,
    # reward_clipped = reward_clipped,
    # reward_enter_target_area = reward_enter_target_area,
    # reward_exit_target_area = reward_exit_target_area
)

video_folder = f"DigitsPuzzle-{n_digits}"
record_freq = 200

env = RecordVideo(env, video_folder=f"videos/{video_folder}", name_prefix="training",
                  episode_trigger=lambda x: x % record_freq == 0)

model = A2C(
    "MultiInputPolicy", 
    env, 
    ent_coef=0.01,
    n_steps=64,
    verbose=1, 
    device="mps", 
    tensorboard_log=f"./logs/a2c_digitspuzzle_nd{n_digits}/"
)
model.learn(total_timesteps=1_000_000)

/Users/bapa/miniconda3/envs/gym/lib/python3.12/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /Users/bapa/Codes/KidsPuzzles/videos/DigitsPuzzle-10 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Using mps device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./logs/a2c_digitspuzzle_nd10/A2C_9
MoviePy - Building video /Users/bapa/Codes/KidsPuzzles/videos/DigitsPuzzle-10/training-episode-0.mp4.
MoviePy - Writing video /Users/bapa/Codes/KidsPuzzles/videos/DigitsPuzzle-10/training-episode-0.mp4



MoviePy - Done !
MoviePy - video ready /Users/bapa/Codes/KidsPuzzles/videos/DigitsPuzzle-10/training-episode-0.mp4


-----------------------------------------
| rollout/              |               |
|    ep_len_mean        | 300           |
|    ep_rew_mean        | -104          |
| time/                 |               |
|    fps                | 222           |
|    iterations         | 100           |
|    time_elapsed       | 28            |
|    total_timesteps    | 6400          |
| train/                |               |
|    entropy_loss       | -3.61         |
|    explained_variance | -0.0041235685 |
|    learning_rate      | 0.0007        |
|    n_updates          | 99            |
|    policy_loss        | -9.78         |
|    value_loss         | 9.36          |
-----------------------------------------
------------------------------------------
| rollout/              |                |
|    ep_len_mean        | 300            |
|    ep_rew_mean        | -104           |
| time/                 |                |
|    fps                | 243            |
|    iterations         | 20

MoviePy - Done !
MoviePy - video ready /Users/bapa/Codes/KidsPuzzles/videos/DigitsPuzzle-10/training-episode-200.mp4


...

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 114      |
|    ep_rew_mean        | -11      |
| time/                 |          |
|    fps                | 266      |
|    iterations         | 14800    |
|    time_elapsed       | 3549     |
|    total_timesteps    | 947200   |
| train/                |          |
|    entropy_loss       | -2.65    |
|    explained_variance | 0.858449 |
|    learning_rate      | 0.0007   |
|    n_updates          | 14799    |
|    policy_loss        | 1.11     |
|    value_loss         | 2.3      |
------------------------------------
---------------------------------------
| rollout/              |             |
|    ep_len_mean        | 88          |
|    ep_rew_mean        | -9.78       |
| time/                 |             |
|    fps                | 266         |
|    iterations         | 14900       |
|    time_elapsed       | 3572        |
|    total_timesteps    | 953600      |
| train/   

MoviePy - Done !
MoviePy - video ready /Users/bapa/Codes/KidsPuzzles/videos/DigitsPuzzle-10/training-episode-3600.mp4


-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 91.7      |
|    ep_rew_mean        | -9.82     |
| time/                 |           |
|    fps                | 266       |
|    iterations         | 15200     |
|    time_elapsed       | 3644      |
|    total_timesteps    | 972800    |
| train/                |           |
|    entropy_loss       | -2.4      |
|    explained_variance | 0.8065046 |
|    learning_rate      | 0.0007    |
|    n_updates          | 15199     |
|    policy_loss        | -1.62     |
|    value_loss         | 0.852     |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 79.8      |
|    ep_rew_mean        | -9.11     |
| time/                 |           |
|    fps                | 266       |
|    iterations         | 15300     |
|    time_elapsed       | 3668      |
|    total_timesteps    | 979200    |
| train/    

MoviePy - Done !
MoviePy - video ready /Users/bapa/Codes/KidsPuzzles/videos/DigitsPuzzle-10/training-episode-3800.mp4


-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 81.2      |
|    ep_rew_mean        | -9.44     |
| time/                 |           |
|    fps                | 266       |
|    iterations         | 15400     |
|    time_elapsed       | 3692      |
|    total_timesteps    | 985600    |
| train/                |           |
|    entropy_loss       | -2.48     |
|    explained_variance | 0.8735326 |
|    learning_rate      | 0.0007    |
|    n_updates          | 15399     |
|    policy_loss        | 4.6       |
|    value_loss         | 4.57      |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 83.2      |
|    ep_rew_mean        | -9.35     |
| time/                 |           |
|    fps                | 266       |
|    iterations         | 15500     |
|    time_elapsed       | 3716      |
|    total_timesteps    | 992000    |
| train/    

In [2]:
import time
vec_env = model.get_env()
observations = vec_env.reset()
reward_sum = 0
for _ in range(50):
    action, _state = model.predict(observations)
    observations, reward, terminated, info = vec_env.step(action)
    vec_env.render("human")
    time.sleep(5)

    reward_sum += reward
    if terminated:
        observations = vec_env.reset()

print("Total reward: ", reward_sum)

2025-02-05 08:22:15.923 python[4116:165327] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-05 08:22:15.923 python[4116:165327] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Total reward:  [-6.7222214]


In [3]:
env.close()

MoviePy - Building video /Users/bapa/Codes/KidsPuzzles/videos/DigitsPuzzle-10/training-episode-4000.mp4.
MoviePy - Writing video /Users/bapa/Codes/KidsPuzzles/videos/DigitsPuzzle-10/training-episode-4000.mp4



MoviePy - Done !
MoviePy - video ready /Users/bapa/Codes/KidsPuzzles/videos/DigitsPuzzle-10/training-episode-4000.mp4
